In [1]:
import numpy as np
import torch as th
import syft as sy

In [2]:
sy.create_sandbox(globals())

Setting up Sandbox...
	- Hooking PyTorch
	- Creating Virtual Workers:
		- bob
		- theo
		- jason
		- alice
		- andy
		- jon
	Storing hook and workers as global variables...
	Loading datasets from SciKit Learn...
		- Boston Housing Dataset
		- Diabetes Dataset
		- Breast Cancer Dataset
	- Digits Dataset
		- Iris Dataset
		- Wine Dataset
		- Linnerud Dataset
	Distributing Datasets Amongst Workers...
Done!


**Example data**: the correct $\beta$ is $[1, 2, -1]$

In [3]:
X = th.tensor(10 * np.random.randn(30, 3))
y = (X[:, 0] + 2 * X[:, 1] - X[:, 2]).reshape(-1, 1)

Split the data into chunks and send a chunk to each worker, storing pointers to chunks in two `MultiPointerTensor`s.

In [4]:
workers = [alice, bob, theo]
crypto_provider = jon
chunk_size = int(X.shape[0] / len(workers))

def _get_chunk_pointers(data, chunk_size, workers):
    return [
        data[(i * chunk_size):((i+1)*chunk_size), :].send(worker)
        for i, worker in enumerate(workers)
    ] 

X_ptrs = sy.MultiPointerTensor(
    children=_get_chunk_pointers(X, chunk_size, workers))
y_ptrs = sy.MultiPointerTensor(
    children=_get_chunk_pointers(y, chunk_size, workers))

This is the "big data" step, and it's performed locally on each worker in plain text. The result is two `MultiPointerTensor`s with pointers to each workers' summand of $X^tX$ (or $X^ty$).

In [5]:
Xt_ptrs = X_ptrs.transpose(0, 1)

XtX_summand_ptrs = Xt_ptrs.mm(X_ptrs)
Xty_summand_ptrs = Xt_ptrs.mm(y_ptrs)

We add those summands up in two steps:
- share each summand among all other workers
- move the resulting pointers to one place (here just the local worker) and add 'em up.

In [6]:
def _generate_shared_summand_pointers(
        summand_ptrs, 
        workers, 
        crypto_provider):

    for worker_id, summand_pointer in summand_ptrs.child.items():
        shared_summand_pointer = summand_pointer.fix_precision().share(
            *workers, crypto_provider=crypto_provider)
        yield shared_summand_pointer.get()

In [7]:
XtX_shared = sum(
    _generate_shared_summand_pointers(
        XtX_summand_ptrs, workers, crypto_provider))

Xty_shared = sum(_generate_shared_summand_pointers(
    Xty_summand_ptrs, workers, crypto_provider))

The coefficient $\beta$ is the solution to
$$X^t X \beta = X^t y$$

**TODO**: It _should_ be possible to solve by performing additive operations on the `XtX_shared` and `Xty_shared` (seems like Gaussian elimination or LDL decomposition could be implemented using operations that `AdditiveSharingTensor` supports). In that case, we'd perform those operations and reconstruct $\beta$ on each worker without the local worker having to see any sensitive values.

For now, we're going to punt and just move `XtX_shared` and `Xty_shared` to the local worker. So there's one party that's trusted to see $X^t X$ and $X^t y$, which are effectively summary statistics of the data.


In [8]:
beta = XtX_shared.get().float_precision().inverse().mm(
    Xty_shared.get().float_precision())

In [9]:
beta

tensor([[ 1.0000],
        [ 2.0000],
        [-1.0000]])